# 03_report_export_notify
Export generated views to CSV/Parquet and notify stakeholders.

In [ ]:
import json, os
from pathlib import Path
from datetime import datetime
import requests
import smtplib
from email.message import EmailMessage

catalog='finance'
schema='kyc_ml'
export_dir = '/dbfs/tmp/reports'
Path(export_dir).mkdir(parents=True, exist_ok=True)

with open('/dbfs/FileStore/report_metadata/report_definitions.json','r') as f:
    reports = json.load(f)

def notify_slack(message):
    try:
        webhook = dbutils.secrets.get(scope='notify', key='slack_webhook')
    except Exception:
        webhook = None
    if webhook:
        resp = requests.post(webhook, json={'text': message})
        print('Slack status', resp.status_code)
    else:
        print('Slack not configured. Message:', message)

def send_email(subject, body, to_addrs, attachments=None):
    try:
        smtp_host = dbutils.secrets.get(scope='notify', key='smtp_host')
        smtp_port = int(dbutils.secrets.get(scope='notify', key='smtp_port'))
        smtp_user = dbutils.secrets.get(scope='notify', key='smtp_user')
        smtp_pass = dbutils.secrets.get(scope='notify', key='smtp_pass')
    except Exception as e:
        print('SMTP secrets not configured:', e)
        return
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = smtp_user
    msg['To'] = ','.join(to_addrs)
    msg.set_content(body)
    attachments = attachments or []
    for a in attachments:
        with open(a, 'rb') as f:
            data = f.read()
        msg.add_attachment(data, maintype='application', subtype='octet-stream', filename=Path(a).name)
    with smtplib.SMTP(smtp_host, smtp_port) as s:
        s.starttls()
        s.login(smtp_user, smtp_pass)
        s.send_message(msg)
    print('Email sent to', to_addrs)

exported = []
for rpt in reports:
    view = f"{catalog}.{schema}.{rpt['report_name']}"
    try:
        df = spark.table(view)
        name = rpt['report_name']
        ts = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
        csv_path = f"{export_dir}/{name}_{ts}.csv"
        parquet_path = f"{export_dir}/{name}_{ts}.parquet"
        df.coalesce(1).write.mode('overwrite').option('header',True).csv(csv_path)
        df.write.mode('overwrite').parquet(parquet_path)
        print('Exported', name)
        exported.append({'name':name,'csv':csv_path,'parquet':parquet_path,'notify':rpt.get('notify',[])})
    except Exception as e:
        print('Export failed for', view, e)

for item in exported:
    msg = f"Report {item['name']} exported to {item['csv']}"
    notify_slack(msg)
    if item['notify']:
        send_email(f"Report: {item['name']}", msg, item['notify'], attachments=[])
print('Export & notify complete.')